In [1]:
#| default_exp models.oakY

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#| export
import torch, re, inspect, pickle, os, torch.nn as nn, math
import torch.nn.functional as F
from dataclasses import dataclass
from typing import Optional, List, Tuple, Mapping, Any, Union
from transformers import (
    PretrainedConfig,
    DistilBertForMaskedLM,
    DistilBertModel,
    DistilBertPreTrainedModel,
    DistilBertConfig,
)
from transformers.utils.generic import ModelOutput
from transformers.activations import get_activation

from fastcore.meta import *
from fastcore.utils import *

from xcai.losses import *
from xcai.core import store_attr
from xcai.learner import XCDataParallel
from xcai.models.modeling_utils import *

In [4]:
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [5]:
from transformers import AutoConfig
from xcai.block import *

## Setup

In [67]:
from xcai.main import *

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

In [64]:
data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/'
config_file = 'wikiseealsotitles'
config_key = 'data_lnk'

mname = 'sentence-transformers/msmarco-distilbert-base-v4'

In [65]:
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets/processed/'
pkl_file = f'{pkl_dir}/wikiseealsotitles_data-meta_distilbert-base-uncased_sxc.joblib'

In [68]:
block = build_block(pkl_file, config_file, True, config_key, data_dir=data_dir, n_sdata_meta_samples=3, do_build=False)

/home/scai/phd/aiz218323/.local/lib/python3.10/site-packages/xclib-0.97-py3.10-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [69]:
batch = block.train.one_batch(5)
for i,batch in enumerate(block.train.dl):
    if i > 2: break
    

In [70]:
batch.keys()

dict_keys(['data_idx', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_data2ptr', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_data2ptr', 'lnk2data_identifier', 'lnk2data_input_text', 'lnk2data_input_ids', 'lnk2data_attention_mask', 'plnk2lbl_idx', 'plnk2lbl_lbl2ptr', 'lnk2lbl_idx', 'lnk2lbl_lbl2ptr', 'lnk2lbl_identifier', 'lnk2lbl_input_text', 'lnk2lbl_input_ids', 'lnk2lbl_attention_mask', 'lnk2lbl_data2ptr', 'plnk2lbl_data2ptr'])

## Encoder

In [71]:
#| export
class Encoder(DistilBertPreTrainedModel):
    
    def __init__(
        self, 
        config:PretrainedConfig, 
    ):
        super().__init__(config)
        self.distilbert = DistilBertModel(config)
        
        self.dr_head = RepresentationHead(config)
        self.dr_fused_head = RepresentationHead(config)
        self.meta_head = RepresentationHead(config)
        self.cross_head = CrossAttention(config)
        self.meta_embeddings = nn.Embedding(config.vocab_size, config.dim)
        
        self.post_init()

    def freeze_meta_embeddings(self):
        self.meta_embeddings.requires_grad_(False)

    def unfreeze_meta_embeddings(self):
        self.meta_embeddings.requires_grad_(True)

    def set_meta_embeddings(self, embed:torch.Tensor):
        with torch.no_grad():
            self.meta_embeddings.weight.copy_(embed)
        
    def get_position_embeddings(self) -> nn.Embedding:
        return self.distilbert.get_position_embeddings()
    
    def resize_position_embeddings(self, new_num_position_embeddings: int):
        self.distilbert.resize_position_embeddings(new_num_position_embeddings)
    
    def encode(self, input_ids:torch.Tensor, attention_mask:torch.Tensor, **kwargs):
        return self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
    
    def dr(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.dr_head(embed)
        return F.normalize(Pooling.mean_pooling(embed, attention_mask), dim=1)

    def dr_fused(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.dr_fused_head(embed)
        return F.normalize(Pooling.mean_pooling(embed, attention_mask), dim=1)

    def meta(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.meta_head(embed)
        return F.normalize(Pooling.mean_pooling(embed, attention_mask), dim=1)
    
    def meta_unnormalized(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.meta_head(embed)
        return Pooling.mean_pooling(embed, attention_mask)

    def fuse_meta_into_embeddings(self, embed:torch.Tensor, attention_mask:torch.Tensor, meta_kwargs:Dict):
        meta_repr = {}
        
        for m_key, m_args in meta_kwargs.items():
            idx = torch.where(m_args['data2ptr'] > 0)[0]
            meta_repr[m_key] = torch.empty(0, self.config.dim).to(embed)
            
            if len(idx):
                assert torch.all(m_args['data2ptr'][idx] == m_args['data2ptr'].max()), f'All datapoints should have same number of metadata.'
                
                if 'meta_repr' in m_args:
                    m_repr,m_repr_mask = m_args['meta_repr'],torch.any(m_args['attention_mask'], dim=1).long().view(-1,1)
                    m_repr_mask = m_repr_mask.bool()
                else:
                    m_input_ids, m_attention_mask = m_args['input_ids'], m_args['attention_mask']
                    m_embed = self.meta_embeddings(m_input_ids)
    
                    m_repr = self.meta_unnormalized(m_embed, m_attention_mask)
                    m_repr_mask = torch.any(m_attention_mask, dim=1)
                    
                m_repr, m_repr_mask = m_repr.view(len(idx), -1, self.config.dim), m_repr_mask.view(len(idx), -1)
                
                meta_repr[m_key] = m_repr[m_repr_mask]
                meta_repr[m_key] = F.normalize(meta_repr[m_key], dim=1)
                
                fused_embed, w = self.cross_head(embed[idx], attention_mask[idx], m_repr, m_repr_mask, output_attentions=True)
                embed[idx] += fused_embed
               
        return embed, meta_repr

    def forward(
        self, 
        data_input_ids: torch.Tensor, 
        data_attention_mask: torch.Tensor,
        data_aug_meta_prefix: Optional[str]=None,
        data_type:Optional[str]=None,
        data_unnormalized:Optional[bool]=False,
        **kwargs
    ):  
        data_o = self.encode(data_input_ids, data_attention_mask)
        
        if data_type is not None and data_type == "meta":
            data_repr = self.meta_unnormalized(data_o[0], data_attention_mask) if data_unnormalized else self.meta(data_o[0], data_attention_mask)
        else: 
            data_repr = self.dr(data_o[0], data_attention_mask)
        
        data_fused_repr = meta_repr = None
        if data_aug_meta_prefix is not None:
            meta_kwargs = Parameters.from_meta_aug_prefix(data_aug_meta_prefix, **kwargs)
            if len(meta_kwargs):
                data_fused_embed, meta_repr = self.fuse_meta_into_embeddings(data_o[0], 
                                                                            data_attention_mask, 
                                                                            meta_kwargs)
                data_fused_repr = self.dr_fused(data_fused_embed, data_attention_mask)
                
        return EncoderOutput(
            rep=data_repr,
            fused_rep=data_fused_repr,
            meta_repr=meta_repr,
        )
        

## `OAK000`

In [72]:
#| export
class OAK000(nn.Module):
    
    def __init__(
        self, config,

        data_aug_meta_prefix:Optional[str]=None, 
        lbl2data_aug_meta_prefix:Optional[str]=None, 

        data_pred_meta_prefix:Optional[str]=None,
        lbl2data_pred_meta_prefix:Optional[str]=None,
        
        num_batch_labels:Optional[int]=None, 
        batch_size:Optional[int]=None,
        margin:Optional[float]=0.3,
        num_negatives:Optional[int]=5,
        tau:Optional[float]=0.1,
        apply_softmax:Optional[bool]=True,

        calib_margin:Optional[float]=0.3,
        calib_num_negatives:Optional[int]=10,
        calib_tau:Optional[float]=0.1,
        calib_apply_softmax:Optional[bool]=False,
        calib_loss_weight:Optional[float]=0.1,
        use_calib_loss:Optional[float]=False,
        
        meta_loss_weight:Optional[Union[List,float]]=0.3,
        
        use_fusion_loss:Optional[bool]=False,
        fusion_loss_weight:Optional[float]=0.15,

        use_query_loss:Optional[float]=True,
        
        use_encoder_parallel:Optional[bool]=True,
    ):
        super().__init__(config)
        store_attr('meta_loss_weight,fusion_loss_weight,calib_loss_weight')
        store_attr('data_pred_meta_prefix,lbl2data_pred_meta_prefix')
        store_attr('data_aug_meta_prefix,lbl2data_aug_meta_prefix')
        store_attr('use_fusion_loss,use_query_loss,use_calib_loss,use_encoder_parallel')
        
        self.encoder = None
        self.rep_loss_fn = MultiTriplet(bsz=batch_size, tn_targ=num_batch_labels, margin=margin, n_negatives=num_negatives, 
                                        tau=tau, apply_softmax=apply_softmax, reduce='mean')
        self.cab_loss_fn = Calibration(margin=calib_margin, tau=calib_tau, n_negatives=calib_num_negatives, 
                                       apply_softmax=calib_apply_softmax, reduce='mean')
        
    def init_retrieval_head(self):
        assert self.encoder is not None, 'Encoder is not initialized.'
        self.encoder.dr_head.post_init()
        self.encoder.meta_head.post_init()
        self.encoder.dr_fused_head.post_init()

    def init_cross_head(self):
        assert self.encoder is not None, 'Encoder is not initialized.'
        self.encoder.cross_head.post_init()
        
    def compute_loss(self, inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx):
        return self.rep_loss_fn(inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx)

    def calibration_loss(self, einp_repr, inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx):
        return self.calib_loss_weight * self.cab_loss_fn(einp_repr, inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx)
    
    def compute_meta_loss(self, data_repr, lbl2data_repr, **kwargs):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
            
        data_meta_inputs = Parameters.from_meta_pred_prefix(self.data_pred_meta_prefix, **kwargs)
        lbl2data_meta_inputs = Parameters.from_meta_pred_prefix(self.lbl2data_pred_meta_prefix, **kwargs)
        meta_inputs = {**data_meta_inputs, **lbl2data_meta_inputs}

        m_lw = Parameters.get_meta_loss_weights(self.meta_loss_weight, len(meta_inputs)) if len(meta_inputs) else []
        
        loss = 0.0
        for inputs,lw in zip(meta_inputs.values(), m_lw):
            if 'lbl2data2ptr' in inputs:
                idx = torch.where(inputs['lbl2data2ptr'])[0]
                if len(idx) > 0:
                    inputs_o = encoder(data_input_ids=inputs['input_ids'], data_attention_mask=inputs['attention_mask'], 
                                       data_type="meta")
                    m_loss = self.rep_loss_fn(lbl2data_repr[idx], inputs_o.rep, inputs['lbl2data2ptr'][idx],
                                              inputs['idx'], inputs['plbl2data2ptr'][idx], inputs['pidx'])
                    loss += lw * m_loss

            elif 'data2ptr' in inputs:
                idx = torch.where(inputs['data2ptr'])[0]
                if len(idx) > 0:
                    inputs_o = encoder(data_input_ids=inputs['input_ids'], data_attention_mask=inputs['attention_mask'], 
                                       data_type="meta")
                    m_loss = self.rep_loss_fn(data_repr[idx], inputs_o.rep, inputs['data2ptr'][idx], inputs['idx'], 
                                              inputs['pdata2ptr'][idx], inputs['pidx'])
                    loss += lw * m_loss       

            else: raise ValueError('Invalid metadata input arguments.')
        return loss

    def compute_fusion_loss(self, data_repr, meta_repr:Dict, prefix:str, **kwargs):
        meta_inputs = Parameters.from_meta_pred_prefix(prefix, **kwargs)
        
        loss = 0.0
        if meta_repr is not None:
            for key,input_repr in meta_repr.items():
                inputs = meta_inputs[key]
                if 'lbl2data2ptr' in inputs:
                    idx = torch.where(inputs['lbl2data2ptr'])[0]
                    if len(idx) > 0:
                        m_loss = self.rep_loss_fn(data_repr[idx], input_repr, inputs['lbl2data2ptr'][idx],
                                                  inputs['idx'], inputs['plbl2data2ptr'][idx], inputs['pidx'])
                        loss += self.fusion_loss_weight * m_loss
    
                elif 'data2ptr' in inputs:
                    idx = torch.where(inputs['data2ptr'])[0]
                    if len(idx) > 0:
                        m_loss = self.rep_loss_fn(data_repr[idx], input_repr, inputs['data2ptr'][idx], inputs['idx'], 
                                                  inputs['pdata2ptr'][idx], inputs['pidx'])
                        loss += self.fusion_loss_weight * m_loss       
    
                else: raise ValueError('Invalid metadata input arguments.')
        return loss


    def get_meta_representation(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        **kwargs
    ):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
            
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_unnormalized=True, data_type="meta")
        return XCModelOutput(
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
        )

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)
        
        
        loss = None; lbl2data_o = EncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)
            
            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)

            if self.use_query_loss:
                loss += self.compute_loss(data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                          plbl2data_data2ptr,plbl2data_idx)

            if self.use_calib_loss:
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
            
            # loss += self.compute_meta_loss(data_o.fused_rep, lbl2data_o.rep, **kwargs)
            
            # if self.use_fusion_loss:
            #    loss += self.compute_fusion_loss(data_o.fused_rep, data_o.meta_repr, self.data_aug_meta_prefix, **kwargs)
            #    loss += self.compute_fusion_loss(lbl2data_o.rep, lbl2data_o.meta_repr, self.lbl2data_aug_meta_prefix, **kwargs)
            
            
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o
        
        
        return XCModelOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
        

## `OAK001`

In [ ]:
#| export
class OAK001(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK000.__init__)
    def __init__(self, config, **kwargs):
        super().__init__(config, **kwargs)
        self.encoder = Encoder(config)
        self.post_init(); self.remap_post_init();

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert
        

### Example

In [ ]:
model = OAK001.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=True, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.3, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)
model.init_retrieval_head()
model.init_cross_head()
model.encoder.set_meta_embeddings(model.distilbert.embeddings.word_embeddings.weight)

Some weights of OAK001 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [ ]:
batch.keys()

dict_keys(['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_identifier', 'lnk2data_input_text', 'lnk2data_input_ids', 'lnk2data_attention_mask', 'lnk2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'data_idx', 'lnk2lbl2data_idx', 'lnk2lbl2data_identifier', 'lnk2lbl2data_input_text', 'lnk2lbl2data_input_ids', 'lnk2lbl2data_attention_mask', 'lnk2lbl2data_data2ptr', 'lnk2lbl2data_lbl2data2ptr'])

In [ ]:
o = model(**b.to(model.device))

/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:22: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [ ]:
o.loss

tensor(0.0289, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
def func():
    import pdb; pdb.set_trace()
    return model(**b.to(model.device))
    

In [ ]:
o = func()

> /tmp/ipykernel_28793/3657616883.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return model(**b.to(model.device))
      4 



ipdb>  b model.encoder.cross_head.forward


Breakpoint 5 at /scratch/scai/phd/aiz218323/Projects/xcai/xcai/models/modeling_utils.py:75


ipdb>  c


> /scratch/scai/phd/aiz218323/Projects/xcai/xcai/models/modeling_utils.py(83)forward()
     81         output_attentions:Optional[bool] = False,
     82     ):
---> 83         bs, q_len, dim = q.size()
     84         v, k_len = k, k.size(1)
     85 



...

--Return--
(tensor([[[ 4....iewBackward0>), tensor([[[[0....maxBackward0>))
> /scratch/scai/phd/aiz218323/Projects/xcai/xcai/models/modeling_utils.py(109)forward()
    107         o = self.o(unshape(torch.matmul(w, v))) # (bs, q_len, dim)
    108 
--> 109         if output_attentions: return (o, w)
    110         else: return (o,)
    111 



ipdb>  


--Call--
> /tmp/ipykernel_28793/505101603.py(46)dr_fused()
     44         return F.normalize(Pooling.mean_pooling(embed, attention_mask), dim=1)
     45 
---> 46     def dr_fused(self, embed:torch.Tensor, attention_mask:torch.Tensor):
     47         embed = self.dr_fused_head(embed)
     48         return F.normalize(Pooling.mean_pooling(embed, attention_mask), dim=1)



ipdb>  c


    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



## `OAK002`

In [ ]:
#| export
class Encoder002(Encoder):
    
    def __init__(
        self, 
        config:PretrainedConfig, 
    ):
        super().__init__(config)
        self.post_init()

    def fuse_meta_into_embeddings(self, embed:torch.Tensor, attention_mask:torch.Tensor, meta_kwargs:Dict):
        meta_repr = {}
        
        for m_key, m_args in meta_kwargs.items():
            idx = torch.where(m_args['data2ptr'] > 0)[0]
            meta_repr[m_key] = torch.empty(0, self.config.dim).to(embed)
            
            if len(idx):
                assert torch.all(m_args['data2ptr'][idx] == m_args['data2ptr'].max()), f'All datapoints should have same number of metadata.'
                
                if 'meta_repr' in m_args:
                    m_repr,m_repr_mask = m_args['meta_repr'],torch.any(m_args['attention_mask'], dim=1).long().view(-1,1)
                    m_repr_mask = m_repr_mask.bool()
                else:
                    m_input_ids, m_attention_mask = m_args['input_ids'], m_args['attention_mask']
                    m_embed = self.meta_embeddings(m_input_ids)

                m_repr, m_repr_mask = m_embed.view(len(idx), -1, self.config.dim), m_attention_mask.view(len(idx), -1)
                meta_repr[m_key] = self.meta(m_embed, m_attention_mask)
                
                fused_embed, w = self.cross_head(embed[idx], attention_mask[idx], m_repr, m_repr_mask, output_attentions=True)
                embed[idx] += fused_embed
               
        return embed, meta_repr
        

In [ ]:
#| export
class OAK002(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK000.__init__)
    def __init__(self, config, **kwargs):
        super().__init__(config, **kwargs)
        self.encoder = Encoder002(config)
        self.post_init(); self.remap_post_init();

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert
        

### Example

In [ ]:
model = OAK002.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=10, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=True, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.3, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)
model.init_retrieval_head()
model.init_cross_head()
model.encoder.set_meta_embeddings(model.distilbert.embeddings.word_embeddings.weight)

Some weights of OAK002 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [ ]:
batch.keys()

dict_keys(['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_identifier', 'lnk2data_input_text', 'lnk2data_input_ids', 'lnk2data_attention_mask', 'lnk2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'data_idx', 'lnk2lbl2data_idx', 'lnk2lbl2data_identifier', 'lnk2lbl2data_input_text', 'lnk2lbl2data_input_ids', 'lnk2lbl2data_attention_mask', 'lnk2lbl2data_data2ptr', 'lnk2lbl2data_lbl2data2ptr'])

In [ ]:
o = model(**b.to(model.device))

> /scratch/scai/phd/aiz218323/Projects/xcai/xcai/models/modeling_utils.py(83)forward()
     81         output_attentions:Optional[bool] = False,
     82     ):
---> 83         bs, q_len, dim = q.size()
     84         v, k_len = k, k.size(1)
     85 



ipdb>  c


In [ ]:
o.loss

tensor(0.0132, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
def func():
    import pdb; pdb.set_trace()
    return model(**b.to(model.device))

In [ ]:
o = func()

> /tmp/ipykernel_28793/1795951242.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return model(**b.to(model.device))



ipdb>  b model.encoder.forward


Breakpoint 8 at /tmp/ipykernel_28793/505101603.py:88


ipdb>  c


> /tmp/ipykernel_28793/505101603.py(97)forward()
     95         **kwargs
     96     ):  
---> 97         data_o = self.encode(data_input_ids, data_attention_mask)
     98 
     99         if data_type is not None and data_type == "meta":



...

> /tmp/ipykernel_28793/505101603.py(115)forward()
    113         return EncoderOutput(
    114             rep=data_repr,
--> 115             fused_rep=data_fused_repr,
    116             meta_repr=meta_repr,
    117         )



ipdb>  


> /tmp/ipykernel_28793/505101603.py(113)forward()
    111                 data_fused_repr = self.dr_fused(data_fused_embed, data_attention_mask)
    112 
--> 113         return EncoderOutput(
    114             rep=data_repr,
    115             fused_rep=data_fused_repr,



ipdb>  


    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



In [ ]:
o.loss

tensor(0.0134, grad_fn=<AddBackward0>)

## `OAK003`

In [24]:
#| export
class CrossAttention003(nn.Module):
    
    def __init__(self, config: PretrainedConfig):
        super().__init__()
        self.config, self.n_h, self.dim = config, config.n_heads, config.dim
        self.dropout = nn.Dropout(p=config.attention_dropout)

        if self.dim % self.n_h != 0:
            raise ValueError(f"self.n_heads: {self.n_h} must divide self.dim: {self.dim} evenly.")
            
        self.q = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.k = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.v = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.o = nn.Linear(in_features=config.dim, out_features=config.dim)

    def post_init(self):
        torch.nn.init.eye_(self.q.weight)
        torch.nn.init.eye_(self.k.weight)
        torch.nn.init.eye_(self.v.weight)
        torch.nn.init.eye_(self.o.weight)

    def forward(
        self, 
        q: torch.Tensor,
        k: torch.Tensor, 
        output_attentions:Optional[bool] = False,
    ):  
        bsz, dim, v = q.size(0), q.size(1), k
        h_dim = self.dim//self.n_h

        def shape(x: torch.Tensor): return x.view(bsz, -1, self.n_h, h_dim).transpose(1, 2)
        def unshape(x: torch.Tensor): return x.transpose(1, 2).contiguous().view(bsz, -1, self.n_h * h_dim)

        q = shape(self.q(q))  # (bs, n_h, q_len, h_dim)
        k = shape(self.k(k))  # (bs, n_h, k_len, h_dim)
        v = shape(self.v(v))  # (bs, n_h, k_len, h_dim)

        y = F.scaled_dot_product_attention(q, k, v, is_causal=False)
        o = unshape(y)

        return o.squeeze(1)
        

In [25]:
#| export
class Encoder003(DistilBertPreTrainedModel):
    
    def __init__(
        self, 
        config:PretrainedConfig, 
    ):
        super().__init__(config)
        self.dr_distilbert = DistilBertModel(config)
        self.meta_distilbert = DistilBertModel(config)
        
        self.dr_head = RepresentationHead(config)
        self.cross_head = CrossAttention003(config)
        
        self.post_init()
    
    def dr_encode(self, input_ids:torch.Tensor, attention_mask:torch.Tensor, **kwargs):
        o = self.dr_distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        return Pooling.mean_pooling(o[0], attention_mask)

    def meta_encode(self, input_ids:torch.Tensor, attention_mask:torch.Tensor, **kwargs):
        o = self.meta_distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        return Pooling.mean_pooling(o[0], attention_mask)

    def dr(self, embed:torch.Tensor):
        return F.normalize(self.dr_head(embed), dim=1)

    def init_meta_encoder(self):
        sd_meta, sd_dr = self.meta_distilbert.state_dict(), self.dr_distilbert.state_dict()
        for k in sd_dr:
            assert sd_meta[k].shape == sd_dr[k].shape
            with torch.no_grad():
                sd_meta[k].copy_(sd_dr[k])

    def fuse_meta_into_embeddings(self, embed:torch.Tensor, meta_kwargs:Dict):
        meta_repr, bsz = {}, embed.size(0)
        
        for m_key, m_args in meta_kwargs.items():
            n_meta, valid_meta_idx = m_args['data2ptr'].max(), torch.where(m_args['data2ptr'] > 0)[0]
            assert torch.all(m_args['data2ptr'][valid_meta_idx] == n_meta), f'All datapoints should have same number of metadata.'
            
            m_input_ids, m_attention_mask = m_args['input_ids'], m_args['attention_mask']    
            if m_input_ids.shape[0] > 0: m_embed = self.meta_encode(input_ids=m_input_ids, attention_mask=m_attention_mask)
            else: m_embed = torch.zeros(size=(0,self.config.dim)).to(m_input_ids.device)
            meta_repr[m_key] = m_embed
                            
            m_embed = m_embed.view(len(valid_meta_idx), -1, self.config.dim)  
            fused_embed = self.cross_head(embed[valid_meta_idx], m_embed)
            embed[valid_meta_idx] = embed[valid_meta_idx] + fused_embed
               
        return embed, meta_repr

    def forward(
        self, 
        data_input_ids: torch.Tensor, 
        data_attention_mask: torch.Tensor,
        data_aug_meta_prefix: Optional[str]=None,
        data_type:Optional[str]=None,
        data_unnormalized:Optional[bool]=False,
        **kwargs
    ):  
        data_repr = self.dr_encode(data_input_ids, data_attention_mask)
        
        data_fused_repr = meta_repr = None
        if data_aug_meta_prefix is not None:
            meta_kwargs = Parameters.from_meta_aug_prefix(data_aug_meta_prefix, **kwargs)
            if len(meta_kwargs):
                data_fused_repr, meta_repr = self.fuse_meta_into_embeddings(data_repr, meta_kwargs)
                data_fused_repr = self.dr(data_fused_repr)

        data_repr = F.normalize(data_repr, dim=1)
                
        return EncoderOutput(
            rep=data_repr,
            fused_rep=data_fused_repr,
            meta_repr=meta_repr,
        )
        

In [26]:
#| export
class OAK003(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.dr_distilbert"]
    _keys_to_ignore_on_load_missing = ["encoder.meta_distilbert"]

    @delegates(OAK000.__init__)
    def __init__(self, config, **kwargs):
        super().__init__(config, **kwargs)
        self.encoder = Encoder003(config)
        self.post_init(); self.remap_post_init();

    def init_retrieval_head(self):
        self.encoder.dr_head.post_init()

    def init_meta_encoder(self):
        self.encoder.init_meta_encoder()

    def remap_post_init(self):
        self.distilbert = self.encoder.dr_distilbert
        

### Example

In [ ]:
model = OAK003.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=10, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=True, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.0, 
                               
                               fusion_loss_weight=0.0, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)
model.init_retrieval_head()
model.init_meta_encoder()
model.init_cross_head()

Some weights of OAK003 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [ ]:
batch.keys()

dict_keys(['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_identifier', 'lnk2data_input_text', 'lnk2data_input_ids', 'lnk2data_attention_mask', 'lnk2data_data2ptr', 'lnk2lbl2data_idx', 'lnk2lbl2data_identifier', 'lnk2lbl2data_input_text', 'lnk2lbl2data_input_ids', 'lnk2lbl2data_attention_mask', 'lnk2lbl2data_data2ptr', 'lnk2lbl2data_lbl2data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'data_idx'])

In [ ]:
o = model(**b.to(model.device))

In [ ]:
o.loss

tensor(0.0175, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
def func():
    import pdb; pdb.set_trace()
    return model(**b.to(model.device))

In [ ]:
o = func()

> /tmp/ipykernel_7033/1795951242.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return model(**b.to(model.device))



ipdb>  b model.forward


Breakpoint 6 at /tmp/ipykernel_7033/2804362729.py:140


ipdb>  b model.encoder.forward


Breakpoint 7 at /tmp/ipykernel_7033/1294998010.py:60


...

torch.float32


ipdb>  c


> /tmp/ipykernel_7033/2804362729.py(179)forward()
    177                                           plbl2data_data2ptr,plbl2data_idx)
    178 
--> 179             if self.use_calib_loss:
    180                 loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
    181                                               plbl2data_data2ptr,plbl2data_idx)



ipdb>  c


    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



In [ ]:
sd1 = model.encoder.dr_distilbert.state_dict()
sd2 = model.encoder.meta_distilbert.state_dict()

## `OAK004`

In [ ]:
#| export
class Encoder004(DistilBertPreTrainedModel):
    
    def __init__(
        self, 
        config:PretrainedConfig, 
    ):
        super().__init__(config)
        self.distilbert = DistilBertModel(config)
        self.dr_head = RepresentationHead(config)
        self.cross_head = CrossAttention003(config)
        self.post_init()
    
    def encode(self, input_ids:torch.Tensor, attention_mask:torch.Tensor, **kwargs):
        o = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        return Pooling.mean_pooling(o[0], attention_mask)

    def dr(self, embed:torch.Tensor):
        return F.normalize(self.dr_head(embed), dim=1)

    def fuse_meta_into_embeddings(self, embed:torch.Tensor, meta_kwargs:Dict):
        meta_repr, bsz = {}, embed.size(0)
        
        for m_key, m_args in meta_kwargs.items():
            n_meta, valid_meta_idx = m_args['data2ptr'].max(), torch.where(m_args['data2ptr'] > 0)[0]
            assert torch.all(m_args['data2ptr'][valid_meta_idx] == n_meta), f'All datapoints should have same number of metadata.'
            
            m_embed = m_args['meta_repr']
            meta_repr[m_key] = m_embed
                    
            m_embed = m_embed.view(len(valid_meta_idx), -1, self.config.dim)  
            fused_embed = self.cross_head(embed[valid_meta_idx], m_embed)
            embed[valid_meta_idx] = embed[valid_meta_idx] + fused_embed
               
        return embed, meta_repr
        
    def forward(
        self, 
        data_input_ids: torch.Tensor, 
        data_attention_mask: torch.Tensor,
        data_aug_meta_prefix: Optional[str]=None,
        data_type:Optional[str]=None,
        data_unnormalized:Optional[bool]=False,
        **kwargs
    ):  
        data_repr = self.encode(data_input_ids, data_attention_mask)
        
        data_fused_repr = meta_repr = None
        if data_aug_meta_prefix is not None:
            meta_kwargs = Parameters.from_meta_aug_prefix(data_aug_meta_prefix, **kwargs)
            if len(meta_kwargs):
                data_fused_repr, meta_repr = self.fuse_meta_into_embeddings(data_repr, meta_kwargs)
                data_fused_repr = self.dr(data_fused_repr)

        data_repr = F.normalize(data_repr, dim=1)
                
        return EncoderOutput(
            rep=data_repr,
            fused_rep=data_fused_repr,
            meta_repr=meta_repr,
        )
        

In [ ]:
#| export
class OAK004(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK000.__init__)
    def __init__(self, config, num_metadata:int, **kwargs):
        super().__init__(config, **kwargs)
        self.encoder = Encoder004(config)
        self.meta_embeddings = nn.Embedding(num_metadata, config.dim, sparse=True)
        self.post_init(); self.remap_post_init();

    def init_retrieval_head(self):
        self.encoder.dr_head.post_init()

    def init_meta_embeddings(self):
        torch.nn.init.zeros_(self.meta_embeddings.weight)

    def freeze_meta_embeddings(self):
        self.meta_embeddings.requires_grad_(False)

    def unfreeze_meta_embeddings(self):
        self.meta_embeddings.requires_grad_(True)

    def set_meta_embeddings(self, embed:torch.Tensor):
        with torch.no_grad():
            self.meta_embeddings.weight.copy_(embed)

    def _get_encoder_meta_kwargs(self, feat:str, prefix:str, **kwargs):
        meta_kwargs = Parameters.from_feat_meta_aug_prefix(feat, prefix, **kwargs)
        if f'{prefix}_idx' in meta_kwargs:
            m_idx = meta_kwargs[f'{prefix}_idx']
            if len(m_idx): meta_kwargs[f'{prefix}_meta_repr'] = self.meta_embeddings(m_idx)
        return meta_kwargs

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = self._get_encoder_meta_kwargs('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)
        
        
        loss = None; lbl2data_o = EncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_meta_kwargs = self._get_encoder_meta_kwargs('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)
            
            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)

            if self.use_query_loss:
                loss += self.compute_loss(data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                          plbl2data_data2ptr,plbl2data_idx)

            if self.use_calib_loss:
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
            
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o
        
        
        return XCModelOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
        
        

### Example

In [ ]:
model = OAK004.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=10, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=True, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               num_metadata=block.train.dset.meta['lnk_meta'].n_meta,

                               use_query_loss=True,
                               
                               use_encoder_parallel=False)

model.init_retrieval_head()
model.init_cross_head()

model.set_meta_embeddings(torch.ones(block.train.dset.meta['lnk_meta'].n_meta, model.config.dim))

Some weights of OAK004 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'meta_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_data2ptr',
])

In [ ]:
batch.keys()

dict_keys(['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_identifier', 'lnk2data_input_text', 'lnk2data_input_ids', 'lnk2data_attention_mask', 'lnk2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'data_idx', 'lnk2lbl2data_idx', 'lnk2lbl2data_identifier', 'lnk2lbl2data_input_text', 'lnk2lbl2data_input_ids', 'lnk2lbl2data_attention_mask', 'lnk2lbl2data_data2ptr', 'lnk2lbl2data_lbl2data2ptr'])

In [ ]:
o = model(**b.to(model.device))

In [ ]:
o.loss

In [ ]:
def func():
    import pdb; pdb.set_trace()
    return model(**b.to(model.device))

In [ ]:
o = func()

> /tmp/ipykernel_843/1795951242.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return model(**b.to(model.device))



ipdb>  b model.forward


Breakpoint 1 at /tmp/ipykernel_843/2142014626.py:39


ipdb>  b model.encoder.forward


Breakpoint 2 at /tmp/ipykernel_843/2257606702.py:41


...

ipdb>  c


> /tmp/ipykernel_843/2257606702.py(50)forward()
     48         **kwargs
     49     ):  
---> 50         data_repr = self.encode(data_input_ids, data_attention_mask)
     51 
     52         data_fused_repr = meta_repr = None



ipdb>  c


/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:22: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



## `OAK005`

In [96]:
#| export
class Encoder005(Encoder003):
    
    def __init__(
        self, 
        config:PretrainedConfig, 
    ):
        super().__init__(config)
        self.alpha = nn.Parameter(torch.ones(1))

    def fuse_meta_into_embeddings(self, embed:torch.Tensor, meta_kwargs:Dict):
        meta_repr, bsz = {}, embed.size(0)
        
        for m_key, m_args in meta_kwargs.items():
            n_meta = m_args['data2ptr'].max()
            assert torch.all(m_args['data2ptr'] == n_meta), f'All datapoints should have same number of metadata.'
            
            m_input_ids, m_attention_mask = m_args['input_ids'], m_args['attention_mask']
            m_embed = self.meta_encode(input_ids=m_input_ids, attention_mask=m_attention_mask)
            m_embed = m_embed + m_args['meta_repr'] * self.alpha
            
            meta_repr[m_key] = m_embed
                    
            m_embed = m_embed.view(bsz, -1, self.config.dim)  
            fused_embed = self.cross_head(embed, m_embed)
            embed = embed + fused_embed
               
        return embed, meta_repr
        

In [97]:
#| export
class OAK005(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.dr_distilbert"]
    _keys_to_ignore_on_load_missing = ["encoder.meta_distilbert"]

    @delegates(OAK000.__init__)
    def __init__(self, config, num_metadata:int, num_metadata_clusters:int, do_meta_embed_sparse:Optional[bool]=True, **kwargs):
        super().__init__(config, **kwargs)
        self.encoder = Encoder005(config)
        
        self.meta_embeddings = nn.Embedding(num_metadata_clusters, config.dim, sparse=do_meta_embed_sparse)
        self.register_buffer("metadata_cluster_mapping", torch.arange(num_metadata)%num_metadata_clusters, persistent=True)
        
        self.post_init(); self.remap_post_init();

    def init_meta_embeddings(self):
        torch.nn.init.zeros_(self.meta_embeddings.weight)

    def set_meta_embeddings(self, embed:torch.Tensor):
        with torch.no_grad():
            self.meta_embeddings.weight.copy_(embed)

    def set_metadata_cluster_mapping(self, metadata_cluster_mapping:torch.Tensor):
        if metadata_cluster_mapping.shape[0] != self.metadata_cluster_mapping.shape[0]:
            raise ValueError(f'Shape mismatch, `metadata_cluster_mapping` should have {self.metadata_cluster_mapping.shape[0]} elements.')
        with torch.no_grad():
            self.metadata_cluster_mapping.copy_(metadata_cluster_mapping)

    def init_retrieval_head(self):
        self.encoder.dr_head.post_init()

    def init_meta_encoder(self):
        self.encoder.init_meta_encoder()

    def remap_post_init(self):
        self.distilbert = self.encoder.dr_distilbert

    def _get_encoder_meta_kwargs(self, feat:str, prefix:str, **kwargs):
        meta_kwargs = Parameters.from_feat_meta_aug_prefix(feat, prefix, **kwargs)
        if f'{prefix}_idx' in meta_kwargs:
            m_idx = meta_kwargs[f'{prefix}_idx']
            if len(m_idx): meta_kwargs[f'{prefix}_meta_repr'] = self.meta_embeddings(self.metadata_cluster_mapping[m_idx])
        return meta_kwargs

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = self._get_encoder_meta_kwargs('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)
        
        
        loss = None; lbl2data_o = EncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_meta_kwargs = self._get_encoder_meta_kwargs('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)
            
            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)

            if self.use_query_loss:
                loss += self.compute_loss(data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                          plbl2data_data2ptr,plbl2data_idx)

            if self.use_calib_loss:
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
                
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o
        
        
        return XCModelOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
        

### Example

In [98]:
from xcai.core import prepare_batch

In [99]:
meta_name = 'lnk'

In [100]:
n_clusters = block.train.dset.meta[f'{meta_name}_meta'].n_meta // 3
meta_cluster_mapping = torch.arange(block.train.dset.meta[f'{meta_name}_meta'].n_meta) % n_clusters

In [101]:
model = OAK005.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=10, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix=f'{meta_name}2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,

                               num_metadata=block.train.dset.meta[f'{meta_name}_meta'].n_meta, num_metadata_clusters=n_clusters,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=True, 
                               calib_loss_weight=0.1, use_calib_loss=True,

                               use_query_loss=True,
                               
                               use_encoder_parallel=False, do_meta_embed_sparse=False)
model.init_retrieval_head()
model.init_meta_encoder()
model.init_cross_head()

Some weights of OAK005 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.alpha', 'encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'meta_embeddings.weight', 'metadata_cluster_mapping']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [102]:
model.init_meta_embeddings()
model.set_metadata_cluster_mapping(meta_cluster_mapping)

In [ ]:
model = model.to('cuda')

In [103]:
b = prepare_batch(model, batch, m_args=[
    f'p{meta_name}2data_idx', f'p{meta_name}2data_data2ptr', f'{meta_name}2data_idx', f'{meta_name}2data_input_ids', 
    f'{meta_name}2data_attention_mask', f'{meta_name}2data_data2ptr',
])

In [104]:
batch.keys()

dict_keys(['data_idx', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_data2ptr', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_data2ptr', 'lnk2data_identifier', 'lnk2data_input_text', 'lnk2data_input_ids', 'lnk2data_attention_mask', 'plnk2lbl_idx', 'plnk2lbl_lbl2ptr', 'lnk2lbl_idx', 'lnk2lbl_lbl2ptr', 'lnk2lbl_identifier', 'lnk2lbl_input_text', 'lnk2lbl_input_ids', 'lnk2lbl_attention_mask', 'lnk2lbl_data2ptr', 'plnk2lbl_data2ptr'])

In [106]:
o = model(**b.to(model.device))

> /tmp/ipykernel_15122/1326464437.py(61)forward()
     59         **kwargs
     60     ):  
---> 61         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     62 
     63         if self.use_encoder_parallel:



ipdb>  n


> /tmp/ipykernel_15122/1326464437.py(63)forward()
     61         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     62 
---> 63         if self.use_encoder_parallel:
     64             encoder = XCDataParallel(module=self.encoder)
     65         else: encoder = self.encoder



ipdb>  n


> /tmp/ipykernel_15122/1326464437.py(65)forward()
     63         if self.use_encoder_parallel:
     64             encoder = XCDataParallel(module=self.encoder)
---> 65         else: encoder = self.encoder
     66 
     67         data_meta_kwargs = self._get_encoder_meta_kwargs('data', self.data_aug_meta_prefix, **kwargs)



...

ipdb>  


--Return--
EncoderOutput...eta_repr=None)
> /tmp/ipykernel_15122/1379713280.py(81)forward()
     79         data_repr = F.normalize(data_repr, dim=1)
     80 
---> 81         return EncoderOutput(
     82             rep=data_repr,
     83             fused_rep=data_fused_repr,



ipdb>  


> /tmp/ipykernel_15122/1326464437.py(78)forward()
     76                                  data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)
     77 
---> 78             loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
     79                                      plbl2data_data2ptr,plbl2data_idx)
     80 



ipdb>  c


In [108]:
o.loss

tensor(0.0167, grad_fn=<AddBackward0>)

In [84]:
def func():
    import pdb; pdb.set_trace()
    return model(**b.to(model.device))
    

In [85]:
o = func()

> /tmp/ipykernel_15122/3657616883.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return model(**b.to(model.device))
      4 



ipdb>  b model.forward


Breakpoint 1 at /tmp/ipykernel_15122/1326464437.py:46


ipdb>  b model.encoder.forward


Breakpoint 2 at /tmp/ipykernel_15122/1379713280.py:61


...

> /tmp/ipykernel_15122/1326464437.py(101)forward()
     99 
    100             lbl2data_repr=lbl2data_o.rep,
--> 101             lbl2data_fused_repr=lbl2data_o.fused_rep,
    102         )
    103 



ipdb>  


> /tmp/ipykernel_15122/1326464437.py(94)forward()
     92 
     93 
---> 94         return XCModelOutput(
     95             loss=loss,
     96 



ipdb>  c


    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



In [83]:
o = model(**b.to(model.device))

In [86]:
o.loss

tensor(0.0167, grad_fn=<AddBackward0>)

## `OAK006`

In [ ]:
#| export
class Encoder006(Encoder003):
    
    def __init__(
        self, 
        config:PretrainedConfig,
    ):
        super().__init__(config)

    def dr_encode(self, input_ids:torch.Tensor, attention_mask:torch.Tensor, **kwargs):
        o = self.dr_distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        return F.normalize(Pooling.mean_pooling(o[0], attention_mask), dim=1)

    def meta_encode(self, input_ids:torch.Tensor, attention_mask:torch.Tensor, **kwargs):
        o = self.meta_distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        return F.normalize(Pooling.mean_pooling(o[0], attention_mask), dim=1)

    def fuse_meta_into_embeddings(self, embed:torch.Tensor, meta_kwargs:Dict):
        meta_repr, bsz = {}, embed.size(0)
        
        for m_key, m_args in meta_kwargs.items():
            n_meta = m_args['data2ptr'].max()
            assert torch.all(m_args['data2ptr'] == n_meta), f'All datapoints should have same number of metadata.'
            
            m_input_ids, m_attention_mask = m_args['input_ids'], m_args['attention_mask']
            m_embed = self.meta_encode(input_ids=m_input_ids, attention_mask=m_attention_mask)

            m_alpha = F.sigmoid(m_args['alpha']).unsqueeze(1)
            m_embed = m_alpha * m_embed + (1 - m_alpha) * F.normalize(m_args['meta_repr'], dim=1)
            
            meta_repr[m_key] = m_embed
                    
            m_embed = m_embed.view(bsz, -1, self.config.dim)  
            fused_embed = self.cross_head(embed, m_embed)
            embed = embed + fused_embed
               
        return embed, meta_repr

    def params_from_meta_aug_prefix(self, prefix:str, **kwargs):
        inputs = {}
        args = [arg for arg in kwargs if prefix is not None and re.match(f'^{prefix}.*_(input_ids|attention_mask|data2ptr|meta_repr|idx|alpha)$', arg)]
        for arg in args:
            meta,param = arg.split('_', maxsplit=1)
            inputs.setdefault(meta, {})[param] = kwargs[arg]
        return inputs

    def forward(
        self, 
        data_input_ids: torch.Tensor, 
        data_attention_mask: torch.Tensor,
        data_aug_meta_prefix: Optional[str]=None,
        data_type:Optional[str]=None,
        data_unnormalized:Optional[bool]=False,
        **kwargs
    ):  
        data_repr = self.dr_encode(data_input_ids, data_attention_mask)
        
        data_fused_repr = meta_repr = None
        if data_aug_meta_prefix is not None:
            meta_kwargs = self.params_from_meta_aug_prefix(data_aug_meta_prefix, **kwargs)
            if len(meta_kwargs):
                data_fused_repr, meta_repr = self.fuse_meta_into_embeddings(data_repr, meta_kwargs)
                data_fused_repr = self.dr(data_fused_repr)
                
        return EncoderOutput(
            rep=data_repr,
            fused_rep=data_fused_repr,
            meta_repr=meta_repr,
        )
        

In [ ]:
#| export
class OAK006(OAK005, DistilBertPreTrainedModel):
    
    @delegates(OAK005.__init__)
    def __init__(self, config, num_metadata_clusters:int, **kwargs):
        super().__init__(config, num_metadata_clusters=num_metadata_clusters, **kwargs)
        self.encoder = Encoder006(config)
        
        self.alpha = nn.Parameter(torch.rand(num_metadata_clusters))
        self.post_init(); self.remap_post_init();

    def _get_encoder_meta_kwargs(self, feat:str, prefix:str, **kwargs):
        meta_kwargs = Parameters.from_feat_meta_aug_prefix(feat, prefix, **kwargs)
        if f'{prefix}_idx' in meta_kwargs:
            m_idx = meta_kwargs[f'{prefix}_idx']
            if len(m_idx): 
                meta_kwargs[f'{prefix}_meta_repr'] = self.meta_embeddings(self.metadata_cluster_mapping[m_idx])
                meta_kwargs[f'{prefix}_alpha'] = self.alpha[self.metadata_cluster_mapping[m_idx]]
        return meta_kwargs


### Example

In [ ]:
n_clusters = block.train.dset.meta['lnk_meta'].n_meta // 3
meta_cluster_mapping = torch.arange(block.train.dset.meta['lnk_meta'].n_meta) % n_clusters

In [ ]:
model = OAK006.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=10, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,

                               num_metadata=block.train.dset.meta['lnk_meta'].n_meta, num_metadata_clusters=n_clusters,
                               
                               calib_margin=0.05, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=True,

                               use_query_loss=True,
                               
                               use_encoder_parallel=False)
model.init_retrieval_head()
model.init_meta_encoder()
model.init_cross_head()

Some weights of OAK006 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['alpha', 'encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'meta_embeddings.weight', 'metadata_cluster_mapping']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.init_meta_embeddings()
model.set_metadata_cluster_mapping(meta_cluster_mapping)

In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [ ]:
batch.keys()

dict_keys(['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_identifier', 'lnk2data_input_text', 'lnk2data_input_ids', 'lnk2data_attention_mask', 'lnk2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'data_idx', 'lnk2lbl2data_idx', 'lnk2lbl2data_identifier', 'lnk2lbl2data_input_text', 'lnk2lbl2data_input_ids', 'lnk2lbl2data_attention_mask', 'lnk2lbl2data_data2ptr', 'lnk2lbl2data_lbl2data2ptr'])

In [ ]:
o = model(**b.to(model.device))

In [ ]:
o.loss

tensor(0.0615, grad_fn=<AddBackward0>)

In [ ]:
def func():
    import pdb; pdb.set_trace()
    return model(**b.to(model.device))

In [ ]:
o = func()

> /tmp/ipykernel_6442/1795951242.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return model(**b.to(model.device))



ipdb>  b model.forward


Breakpoint 1 at /tmp/ipykernel_6442/3737515001.py:46


ipdb>  b model.encoder.forward


Breakpoint 2 at /tmp/ipykernel_6442/3083163642.py:55


...

> /tmp/ipykernel_6442/3083163642.py(64)forward()
     62         **kwargs
     63     ):  
---> 64         data_repr = self.dr_encode(data_input_ids, data_attention_mask)
     65 
     66         data_fused_repr = meta_repr = None



ipdb>  n


> /tmp/ipykernel_6442/3083163642.py(66)forward()
     64         data_repr = self.dr_encode(data_input_ids, data_attention_mask)
     65 
---> 66         data_fused_repr = meta_repr = None
     67         if data_aug_meta_prefix is not None:
     68             meta_kwargs = self.params_from_meta_aug_prefix(data_aug_meta_prefix, **kwargs)



ipdb>  c


    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



In [ ]:
o = model(**b.to(model.device))

> /tmp/ipykernel_38498/1476390753.py(61)forward()
     59         **kwargs
     60     ):  
---> 61         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     62 
     63         if self.use_encoder_parallel:



ipdb>  c


> /tmp/ipykernel_38498/1294998010.py(69)forward()
     67         **kwargs
     68     ):  
---> 69         data_repr = self.dr_encode(data_input_ids, data_attention_mask)
     70 
     71         data_fused_repr = meta_repr = None



ipdb>  n


> /tmp/ipykernel_38498/1294998010.py(71)forward()
     69         data_repr = self.dr_encode(data_input_ids, data_attention_mask)
     70 
---> 71         data_fused_repr = meta_repr = None
     72         if data_aug_meta_prefix is not None:
     73             meta_kwargs = Parameters.from_meta_aug_prefix(data_aug_meta_prefix, **kwargs)



...

ipdb>  m_input_ids.shape


torch.Size([15, 14])


ipdb>  m_attention_mask.shape


torch.Size([15, 14])


ipdb>  q


In [ ]:
o.loss

tensor(0.0615, grad_fn=<AddBackward0>)

## `OAK007`

In [ ]:
#| export
class OAK007(OAK003, DistilBertPreTrainedModel):
    
    @delegates(OAK003.__init__)
    def __init__(
        self, 
        config,
        n_labels:int,
        n_clusters:int,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.label_embeddings = nn.Embedding(n_clusters, config.dim)
        self.register_buffer("label_cluster_mapping", torch.arange(n_labels)%n_clusters, persistent=True)
        self.post_init(); self.remap_post_init();

    def init_label_embeddings(self):
        torch.nn.init.zeros_(self.label_embeddings.weight)

    def set_label_embeddings(self, embed:torch.Tensor):
        with torch.no_grad():
            self.label_embeddings.weight.copy_(embed)

    def set_label_cluster_mapping(self, label_cluster_mapping:torch.Tensor):
        if self.label_cluster_mapping.shape[0] != label_cluster_mapping.shape[0]:
            raise ValueError(f'Shape mismatch, `label_cluster_mapping` should have {self.label_cluster_mapping.shape[0]} elements.')
        with torch.no_grad():
            self.label_cluster_mapping.copy_(label_cluster_mapping)

    def get_label_representation(
        self,
        data_idx:Optional[torch.Tensor]=None,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        **kwargs
    ):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask)
        data_o.rep = F.normalize(data_o.rep + self.label_embeddings(self.label_cluster_mapping[data_idx]), dim=1)
        return XCModelOutput(
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
        )

    def forward(
        self,
        data_idx:Optional[torch.Tensor]=None,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)

        loss = None; lbl2data_o = EncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)
            lbl2data_o.rep = F.normalize(lbl2data_o.rep + self.label_embeddings(self.label_cluster_mapping[lbl2data_idx]), dim=1)
            
            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)

            if self.use_query_loss:
                loss += self.compute_loss(data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                          plbl2data_data2ptr,plbl2data_idx)

            if self.use_calib_loss:
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
                
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o

        return XCModelOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
        

### Example

In [ ]:
model = OAK007.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=1000, num_batch_labels=5000, 
                               margin=0.3, num_negatives=10, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=True, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               n_labels=block.n_lbl, n_clusters=block.n_lbl//3,hk

                               use_query_loss=True,
                               
                               use_encoder_parallel=False)
model.init_retrieval_head()
model.init_meta_encoder()
model.init_cross_head()

model.init_label_embeddings()
#model.set_label_cluster_mapping(lbl_remap)

Some weights of OAK007 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'label_cluster_mapping', 'label_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [ ]:
o = model(**b.to(model.device))

/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:22: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [ ]:
o.loss

tensor(0.0610, device='cuda:0', grad_fn=<AddBackward0>)